In [1]:
from env import host, username, password, get_db_url
import os
import pandas as pd 
import numpy as np

In [90]:
def acquire_zillow_data(use_cache=True):
    '''
    This function returns a snippet of zillow's database as a Pandas DataFrame. 
    When this SQL data is cached and extant in the os directory path, return the data as read into a df. 
    If csv is unavailable, aquisition proceeds regardless,
    reading the queried database elements into a dataframe, creating a cached csv file
    and lastly returning the dataframe for some sweet data science perusal.
    '''

    # If the cached parameter is True, read the csv file on disk in the same folder as this file 
    if os.path.exists('zillow.csv') and use_cache:
        print('Using cached CSV')
        return pd.read_csv('zillow.csv', dtype={'buildingclassdesc': 'str', 'propertyzoningdesc': 'str'})

    # When there's no cached csv, read the following query from Codeup's SQL database.
    print('CSV not detected.')
    print('Acquiring data from SQL database instead.')
    df = pd.read_sql(
        '''
        SELECT *
        FROM properties_2017
        JOIN (
              SELECT parcelid, 
                     logerror, 
                     MAX(transactiondate) AS transactiondate 
              FROM predictions_2017 GROUP BY parcelid, logerror
              ) 
               AS pred_2017 USING(parcelid) 
        LEFT JOIN airconditioningtype 
            USING(airconditioningtypeid)
        LEFT JOIN architecturalstyletype 
            USING(architecturalstyletypeid)
        LEFT JOIN buildingclasstype 
            USING(buildingclasstypeid)
        LEFT JOIN heatingorsystemtype 
            USING(heatingorsystemtypeid)
        LEFT JOIN propertylandusetype 
            USING(propertylandusetypeid)
        LEFT JOIN storytype 
            USING(storytypeid)
        LEFT JOIN typeconstructiontype 
            USING(typeconstructiontypeid)
        WHERE properties_2017.latitude IS NOT NULL
        AND properties_2017.longitude IS NOT NULL;             
        '''
                    , get_db_url('zillow'))
    
    df.propertyzoningdesc.astype(str)
    
    
    print('Acquisition Complete. Dataframe available and is now cached for future use.')
    # create a csv of the dataframe for the sake of efficiency. 
    df.to_csv('zillow.csv', index=False)
    
    return df

In [83]:
df = acquire_zillow_data()

Using cached CSV


In [ ]:
df

In [149]:
import mitosheet
mitosheet.sheet(df, view_df=True)

MitoWidget(analysis_data_json='{"analysisName": "UUID-c45336e8-e767-4ab1-98e9-49ef574e4725", "code": {"imports…

In [86]:
df.propertyzoningdesc.astype(str)

dtype('O')

In [79]:
df.architecturalstyledesc.nunique()
print(f'Column: {round(df["architecturalstyledesc"].value_counts(normalize=True).nlargest(5), 3)} \n')
print(f'Column: {round(df["buildingclassdesc"].value_counts(normalize=True).nlargest(5), 3)} \n')
print(f'Column: {round(df["propertyzoningdesc"].value_counts(normalize=True).nlargest(5), 3)} \n')

Column: Contemporary     0.835
Conventional     0.092
Ranch/Rambler    0.034
Bungalow         0.024
Cape Cod         0.015
Name: architecturalstyledesc, dtype: float64 

Column: Buildings having wood or wood and steel frames                                                                                                                                                                                             0.933
Buildings having exterior walls built of a non-combustible material such as brick, concrete, block or poured concrete. Interior partitions and roof structures are built of combustible materials. Floor may be concrete or wood frame.    0.067
Name: buildingclassdesc, dtype: float64 

Column: LAR1       0.134
LAR3       0.049
LARS       0.028
LBR1N      0.024
LARD1.5    0.022
Name: propertyzoningdesc, dtype: float64 



In [99]:
def describe_data(df):
    '''
    This function takes in a pandas dataframe and prints out the shape, datatypes, number of missing values, 
    columns and their data types, summary statistics of numeric columns in the dataframe, as well as the value counts for categorical variables.
    '''
    # Print out the "shape" of our dataframe - rows and columns
    print(f'This dataframe has {df.shape[0]} rows and {df.shape[1]} columns.')
    print('')
    print('--------------------------------------')
    print('--------------------------------------')
    
    # print the datatypes and column names with non-null counts
    print(df.info())
    print('')
    print('--------------------------------------')
    print('--------------------------------------')
    
    
    # print out summary stats for our dataset
    print('Here are the summary statistics of our dataset')
    print(df.describe().applymap(lambda x: f"{x:0.3f}"))
    print('')
    print('--------------------------------------')
    print('--------------------------------------')

    # print the number of missing values per column and the total
    print('Null Values by Column: ')
    missing_total = df.isnull().sum().sum()
    missing_count = df.isnull().sum() # the count of missing values
    value_count = df.isnull().count() # the count of all values
    missing_percentage = round(missing_count / value_count * 100, 2) # percentage of missing values
    missing_df = pd.DataFrame({'count': missing_count, 'percentage': missing_percentage})\
    .sort_values(by='percentage', ascending=False)
    
    print(missing_df.head(50))
    print(f' \n Total Number of Missing Values: {missing_total} \n')
    df_total = df[df.columns[:]].count().sum()
    proportion_of_nulls = round((missing_total / df_total), 4)
    print(f' Proportion of Nulls in Dataframe: {proportion_of_nulls}\n') 
    print('--------------------------------------')
    print('--------------------------------------')
    
    print('Row-by-Row Nulls')
    print(nulls_by_row(df))
    print('----------------------')
    

   # num_cols = [col for col in df.columns if df[col].dtype != 'object']
   # cat_cols = [col for col in df.columns if col not in num_cols]
   # print('Dataframe value counts ')
   # for col in df.columns:
   #     if col in cat_cols:
   #         print(df[col].value_counts())
   #     else:
   #         # define bins for continuous columns and don't sort them
   #         print(df[col].value_counts(bins=10, sort=False))


    #print('Relative Frequencies: \n')
    ## Display top 5 values of each variable within reasonable limit
    #limit = 25
    #for col in df.columns:
    #    if df[col].nunique() < limit:
    #        print(f'Column: {col} \n {round(df[col].value_counts(normalize=True).nlargest(5), 3)} \n')
    #    else: 
    #        print(f'Column: {col} \n')
    #        print(f'Range of Values: [{df[col].min()} - {df[col].max()}] \n')
    #    print('------------------------------------------')
    #    print('--------------------------------------')
        
        
def nulls_by_col(df):
    '''
    This function  takes in a dataframe of observations and attributes(or columns) and returns a dataframe where each row is an atttribute name, the first column is the 
    number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute.
    '''
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    prcnt_miss = (num_missing / rows * 100)
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 
                                 'percent_rows_missing': prcnt_miss})\
    .sort_values(by='percent_rows_missing', ascending=False)
    return cols_missing.applymap(lambda x: f"{x:0.1f}")

def nulls_by_row(df):
    '''
    This function takes in a dataframe and returns a dataframe with 3 columns: the number of columns missing, percent of columns missing, 
    and number of rows with n columns missing.
    '''
    num_missing = df.isnull().sum(axis = 1)
    prcnt_miss = (num_missing / df.shape[1] * 100)
    rows_missing = pd.DataFrame({'num_cols_missing': num_missing, 
                                 'percent_cols_missing': prcnt_miss})\
    .reset_index()\
    .groupby(['num_cols_missing', 'percent_cols_missing']).count()\
    .rename(index=str, columns={'index': 'num_rows'}).reset_index().set_index('num_cols_missing')\
    .sort_values(by='percent_cols_missing', ascending=False)
    return rows_missing

In [33]:
print(f'Column: {round(df["propertyzoningdesc"].value_counts(normalize=True).nlargest(5), 3)} \n')

In [55]:
def nulls_by_col(df):
    '''
    This function  takes in a dataframe of observations and attributes(or columns) and returns a dataframe where each row is an atttribute name, the first column is the 
    number of rows with missing values for that attribute, and the second column is percent of total rows that have missing values for that attribute.
    '''
    num_missing = df.isnull().sum()
    rows = df.shape[0]
    prcnt_miss = (num_missing / rows * 100)
    cols_missing = pd.DataFrame({'num_rows_missing': num_missing, 
                                 'percent_rows_missing': prcnt_miss})\
    .sort_values(by='percent_rows_missing', ascending=False)
    return cols_missing.applymap(lambda x: f"{x:0.1f}")

In [56]:
null_cols = nulls_by_col(df)

In [60]:
null_cols.head(10)

,num_rows_missing,percent_rows_missing
buildingclassdesc,77560.0,100.0
buildingclasstypeid,77560.0,100.0
finishedsquarefeet13,77533.0,99.9
storydesc,77525.0,99.9
basementsqft,77525.0,99.9
storytypeid,77525.0,99.9
yardbuildingsqft26,77505.0,99.9
fireplaceflag,77403.0,99.8
architecturalstyletypeid,77369.0,99.7
architecturalstyledesc,77369.0,99.7


In [44]:
import mitosheet
mitosheet.sheet(null_cols, view_df=True)

MitoWidget(analysis_data_json='{"analysisName": "UUID-d94ad3f2-8849-4948-94e0-ea39ed315646", "code": {"imports…

In [ ]:
from mitosheet import *; register_analysis('UUID-d94ad3f2-8849-4948-94e0-ea39ed315646')
    
# Sorted percent_rows_missing in null_cols in descending order
null_cols = null_cols.sort_values(by='percent_rows_missing', ascending=False, na_position='last')


In [94]:
describe_data(df)

This dataframe has 77575 rows and 68 columns.

--------------------------------------
--------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77575 entries, 0 to 77574
Data columns (total 68 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   typeconstructiontypeid        222 non-null    float64
 1   storytypeid                   50 non-null     float64
 2   propertylandusetypeid         77575 non-null  float64
 3   heatingorsystemtypeid         49570 non-null  float64
 4   buildingclasstypeid           15 non-null     float64
 5   architecturalstyletypeid      206 non-null    float64
 6   airconditioningtypeid         25006 non-null  float64
 7   parcelid                      77575 non-null  int64  
 8   id                            77575 non-null  int64  
 9   basementsqft                  50 non-null     float64
 10  bathroomcnt                   77575 non-null  float64

In [24]:
zillow.dtypes

typeconstructiontypeid    float64
storytypeid               float64
propertylandusetypeid     float64
heatingorsystemtypeid     float64
buildingclasstypeid       float64
                           ...   
buildingclassdesc          object
heatingorsystemdesc        object
propertylandusedesc        object
storydesc                  object
typeconstructiondesc       object
Length: 68, dtype: object

In [101]:
# Data cleanign functions 

def remove_columns(df, cols_to_remove):
    '''
    This function takes in a pandas dataframe and a list of columns to remove. It drops those columns from the original df and returns the df.
    '''
    df = df.drop(columns=cols_to_remove)
    return df
                 
                 
def handle_missing_values(df, prop_required_column=0.5 , prop_required_row=0.75):
    '''
    This function takes in a pandas datafeame, default proportion of required columns (set to 50%) and proprtion of required rows (set to 75%).
    It drops any rows or columns that contain null values more than the threshold specified from the original dataframe and returns that dataframe.
    '''
    threshold = int(round(prop_required_column * len(df.index), 0))
    df = df.dropna(axis=1, thresh=threshold)
    threshold = int(round(prop_required_row * len(df.columns), 0))
    df = df.dropna(axis=0, thresh=threshold)
    return df

# combined in one function
def data_prep(df, cols_to_remove=[], prop_required_column=0.5, prop_required_row=0.75):
    '''
    This function calls the remove_columns and handle_missing_values to drop columns that need to be removed. It also drops rows and columns that have more 
    missing values than the specified threshold.
    '''
    df = remove_columns(df, cols_to_remove)
    df = handle_missing_values(df, prop_required_column, prop_required_row)
    return df

In [133]:
# to find the difference 
from collections import Counter

def handle_missing_values(df, prop_required_column=0.5 , prop_required_row=0.75):
    '''
    This function takes in a pandas dataframe, default proportion of required columns (set to 50%) and proprtion of required rows (set to 75%).
    It drops any rows or columns that contain null values more than the threshold specified from the original dataframe and returns that dataframe.
    
    Prior to returning that data, it will print statistics and list counts/names of removed columns/row counts 
    '''
    original_cols = df.columns.to_list()
    original_rows = df.shape[0]
    threshold = int(round(prop_required_column * len(df.index), 0))
    df = df.dropna(axis=1, thresh=threshold)
    threshold = int(round(prop_required_row * len(df.columns), 0))
    df = df.dropna(axis=0, thresh=threshold)
    remaining_cols = df.columns.to_list()
    remaining_rows = df.shape[0]
    dropped_col_count = len(original_cols) - len(remaining_cols)
    dropped_cols = list((Counter(original_cols) - Counter(remaining_cols)).elements())
    print(f'The following {dropped_col_count} columns were dropped because they were missing more than {prop_required_column * 100}% of data: \n{dropped_cols}\n')
    dropped_rows = original_rows - remaining_rows
    print(f'{dropped_rows} rows were dropped because they were missing more than {prop_required_row * 100}% of data')
    return df

In [134]:
handle_missing_values(df)

The following 34 columns were dropped because they were missing more than 50.0% of data: 
['typeconstructiontypeid', 'storytypeid', 'buildingclasstypeid', 'architecturalstyletypeid', 'airconditioningtypeid', 'basementsqft', 'decktypeid', 'finishedfloor1squarefeet', 'finishedsquarefeet13', 'finishedsquarefeet15', 'finishedsquarefeet50', 'finishedsquarefeet6', 'fireplacecnt', 'garagecarcnt', 'garagetotalsqft', 'hashottuborspa', 'poolcnt', 'poolsizesum', 'pooltypeid10', 'pooltypeid2', 'pooltypeid7', 'regionidneighborhood', 'threequarterbathnbr', 'yardbuildingsqft17', 'yardbuildingsqft26', 'numberofstories', 'fireplaceflag', 'taxdelinquencyflag', 'taxdelinquencyyear', 'airconditioningdesc', 'architecturalstyledesc', 'buildingclassdesc', 'storydesc', 'typeconstructiondesc']

156 rows were dropped because they were missing more than 75.0% of data


,propertylandusetypeid,heatingorsystemtypeid,parcelid,id,bathroomcnt,bedroomcnt,buildingqualitytypeid,calculatedbathnbr,calculatedfinishedsquarefeet,finishedsquarefeet12,...,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,censustractandblock,logerror,transactiondate,heatingorsystemdesc,propertylandusedesc,bedroom_plus_bathroom
0,261.0,NaN,14297519,1727539,3.5,4.0,NaN,3.5,3100.0,3100.0,...,1023282.0,2016.0,537569.0,11013.72,6.059063e+13,0.025595,2017-01-01,NaN,Single Family Residential,7.5
1,261.0,NaN,17052889,1387261,1.0,2.0,NaN,1.0,1465.0,1465.0,...,464000.0,2016.0,376000.0,5672.48,6.111001e+13,0.055619,2017-01-01,NaN,Single Family Residential,3.0
2,261.0,NaN,14186244,11677,2.0,3.0,NaN,2.0,1243.0,1243.0,...,564778.0,2016.0,479489.0,6488.30,6.059022e+13,0.005383,2017-01-01,NaN,Single Family Residential,5.0
3,261.0,2.0,12177905,2288172,3.0,4.0,8.0,3.0,2376.0,2376.0,...,145143.0,2016.0,36225.0,1777.51,6.037300e+13,-0.103410,2017-01-01,Central,Single Family Residential,7.0
4,266.0,2.0,10887214,1970746,3.0,3.0,8.0,3.0,1312.0,1312.0,...,119407.0,2016.0,45726.0,1533.89,6.037124e+13,0.006940,2017-01-01,Central,Condominium,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77570,266.0,2.0,10833991,2864704,3.0,3.0,8.0,3.0,1741.0,1741.0,...,379000.0,2016.0,114000.0,4685.34,6.037132e+13,-0.002245,2017-09-20,Central,Condominium,6.0
77571,261.0,2.0,11000655,673515,2.0,2.0,6.0,2.0,1286.0,1286.0,...,354621.0,2016.0,283704.0,4478.43,6.037101e+13,0.020615,2017-09-20,Central,Single Family Residential,4.0
77572,261.0,NaN,17239384,2968375,2.0,4.0,NaN,2.0,1612.0,1612.0,...,67205.0,2016.0,16522.0,1107.48,6.111008e+13,0.013209,2017-09-21,NaN,Single Family Residential,6.0
77573,261.0,2.0,12773139,1843709,1.0,3.0,4.0,1.0,1032.0,1032.0,...,49546.0,2016.0,16749.0,876.43,6.037434e+13,0.037129,2017-09-21,Central,Single Family Residential,4.0


In [102]:
df2 = data_prep(df)

In [103]:
describe_data(df2)

This dataframe has 77319 rows and 34 columns.

--------------------------------------
--------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 77319 entries, 0 to 77574
Data columns (total 34 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   propertylandusetypeid         77319 non-null  float64
 1   heatingorsystemtypeid         49570 non-null  float64
 2   parcelid                      77319 non-null  int64  
 3   id                            77319 non-null  int64  
 4   bathroomcnt                   77319 non-null  float64
 5   bedroomcnt                    77319 non-null  float64
 6   buildingqualitytypeid         49806 non-null  float64
 7   calculatedbathnbr             76938 non-null  float64
 8   calculatedfinishedsquarefeet  77315 non-null  float64
 9   finishedsquarefeet12          73905 non-null  float64
 10  fips                          77319 non-null  float64

In [104]:
def remove_outliers(df, k, col_list):
    ''' remove outliers from a list of columns in a dataframe 
        and return that dataframe
    '''
    
    for col in col_list:
        # get quartiles
        q1, q3 = df[f'{col}'].quantile([.25, .75])  
        # calculate interquartile range
        iqr = q3 - q1   
        # get upper bound
        upper_bound = q3 + k * iqr 
        # get lower bound
        lower_bound = q1 - k * iqr   

        # return dataframe without outliers
        
        df = df[(df[f'{col}'] > lower_bound) & (df[f'{col}'] < upper_bound)]
        
    return df

In [110]:
df2.parcelid.nunique
#it's showing 77574...but there are only 77319 rows. What's going on? 

<bound method IndexOpsMixin.nunique of 0        14297519
1        17052889
2        14186244
3        12177905
4        10887214
           ...   
77570    10833991
77571    11000655
77572    17239384
77573    12773139
77574    12826780
Name: parcelid, Length: 77319, dtype: int64>

In [111]:
# remove duplicate parcelids
df2 = df2.sort_values('transactiondate').drop_duplicates('parcelid',keep='last')

In [113]:
df2.parcelid.nunique

<bound method IndexOpsMixin.nunique of 0        14297519
1        17052889
2        14186244
3        12177905
4        10887214
           ...   
77571    11000655
77573    12773139
77572    17239384
77574    12826780
15132    13083743
Name: parcelid, Length: 77130, dtype: int64>

In [114]:
df2.shape
#this had an impact. But, duplicate parcelids simply have to go. 

(77130, 34)

In [115]:
# let's check for more duplicates 
# drop duplicate columns
df2 = df2.loc[:,~df2.columns.duplicated()]

In [116]:
df2.shape
# no other duplicates to worry about. Shape same as before. 

(77130, 34)

While there can be condo townhomes and condo apartments, there cannot be condo-style single family homes. Single-family homes also can be placed on any size of property, which means that single-family homes come in all shapes and sizes.

A single-family home is a free-standing residential building. Single-family homes are designed to be used as a single-dwelling unit, with one owner, no shared walls, and its own land.

In [119]:
df.propertylandusedesc.unique()

array(['Single Family Residential', 'Condominium',
       'Duplex (2 Units, Any Combination)', 'Residential General',
       'Planned Unit Development', 'Triplex (3 Units, Any Combination)',
       'Quadruplex (4 Units, Any Combination)', 'Cluster Home',
       'Commercial/Office/Residential Mixed Used', 'Cooperative',
       'Manufactured, Modular, Prefabricated Homes', 'Mobile Home',
       'Townhouse'], dtype=object)

In [128]:
df2 = df2[(df2.propertylandusedesc == 'Single Family Residential') |
      (df2.propertylandusedesc == 'Mobile Home') |
      (df2.propertylandusedesc == 'Manufactured, Modular, Prefabricated Homes') |
      (df2.propertylandusedesc == 'Cluster Home')]

In [130]:
df2.shape

(52639, 35)

In [127]:
df2["bedroom_plus_bathroom"] = df2.bathroomcnt + df2.bedroomcnt
(df2.bedroom_plus_bathroom == df2.calculatedbathnbr).sum()

256

In [135]:

df2= df2[(df2.bedroomcnt > 0) & (df2.bathroomcnt > 0)]
 
# remove properties with 0 baths
df2 = df2[df2.bathroomcnt > 0]

In [136]:
df2.shape

(52561, 35)

In [137]:
# # keep only properties with square footage greater than 70 (legal size of a bedroom)
df2 = df2[df2.calculatedfinishedsquarefeet > 70]

In [138]:
df2.shape

(52560, 35)

In [139]:
df2 = df2[df2.unitcnt <= 1]

In [140]:
df2.shape

(33743, 35)

In [ ]:
df2

In [142]:
import mitosheet
mitosheet.sheet(df2, view_df=True)

MitoWidget(analysis_data_json='{"analysisName": "UUID-91b1b94e-297e-4c07-9a5c-8ef98874204a", "code": {"imports…

In [ ]:
from mitosheet import *; register_analysis('UUID-91b1b94e-297e-4c07-9a5c-8ef98874204a')
    
# Reordered fullbathcnt in df2
df2_columns = [col for col in df2.columns if col != 'fullbathcnt']
df2_columns.insert(6, 'fullbathcnt')
df2 = df2[df2_columns]


In [143]:
(df2.calculatedfinishedsquarefeet == df2.finishedsquarefeet12).sum()
# this indicated thats finishedsquarefeet12 and calculatedfinishedsquarefeet are identical. drop one. 

33743

In [144]:
(df2.fullbathcnt == df2.bathroomcnt).sum()
# completely identical again. Drop it. 

33743

In [148]:
(df2.bathroomcnt == df2.calculatedbathnbr).sum()

33743